# Tutorial

We now illustrate the basic capabilities of the ``respy`` package in a simple tutorial. The tutorial covers the following topics:

1. [How to specify a model - parameters and options.](#How-to-specify-a-model---parameters-and-options.)
2. [How to solve a model.](#How-to-solve-a-model.)
3. [How to simulate a data set.](#How-to-simulate-a-data-set.)
4. [How to estimate model parameters.](#How-to-estimate-model-parameters.)

After the [installation](installation.rst), the convention is to import ``respy`` as follows to get access to all exposed functions.

In [1]:
import respy as rp

## How to specify a model - parameters and options.

In order to define a model, ``respy`` relies on two objects - parameters and options. The difference between the two is that parameters are affected by the optimization whereas options stay the same. Here, only some parameters and options are explained. For a complete overview on parameters and options, visit the section [model specification](../software/model-specification.rst).

As an example, we turn to the first model of Keane and Wolpin (1994). The following function returns the parameters and options and the original dataset used by the authors. Right now, we do not need the data.

In [2]:
params, options, _ = rp.get_example_model("kw_94_one")

In [3]:
_, df = rp.simulate(params, options)

In [4]:
crit_func = rp.get_crit_func(params, options, df)

/mnt/c/Users/tobia/git/respy/respy/likelihood.py:378: UserWarning: The maximum experience for choice 'b' differ between data, 36, and options, 39. The options are adjusted.
  category=UserWarning,


As said before, options are everything not affected by the optimization. The value under ``"n_periods"`` defines the number of periods in the model meaning the available time frame in which individual can make their decisions.

Choices which are available to individuals within the model are specified under ``"choices"``. An empty dictionary as a value signals that this choice relies on default values generated inside of ``respy`` or does not need information at all. Under ``"edu"`` we can see which options are available for each choice.

- ``"max"`` defines the maximum amount of experience which can be accumulated by individuals using this choice.
- ``"start"`` is a list and contains different initial experience levels of this choice. Here, individuals start with 10 years of experience.
- ``"lagged"`` defines the share of individual with this choice and initial experience level having ``"edu"`` as the initial lagged choice when entering the model.
- ``"share"`` influences the probability of being a particular type.

The rest of the options includes seeds, numbers of draws to simulate the flow utilities in each period and other model parameters. Furthermore, there is a lot of logic hidden from the user to offer a comprehensible starting point. For a deeper understanding of the mechanics, the user is refered to the section [model specfication](../software/model-specification.rst).

The parameter specification includes all parameters of the model which are affected by the optimization routine.

In [ ]:
params.head()

## How to solve a model.

The solution of the model is the value of each state within the model assuming optimal decision in this and future periods. In ``respy``, the solution is represented with the :class:`~respy.state_space.StateSpace`, a class including the model solution. The solution combines the following steps:

- Building the state space.
- Solving the model by finding optimal decision in each state via backward induction.

The ``state_space`` is normally hidden from the user and only an internal construct. For a better understanding of how modeling works, the state space is exposed to the user.

In [ ]:
state_space = rp.solve(params, options)

## How to simulate a data set.

Simulating a data set is as easy as passing model inputs to :func:`~respy.simulate.simulate`. The return values are the ``state_space`` of the model and ``df``, a data set of simulated agents as a :class:`pandas.DataFrame`. The number of simulated agents can be set in the options under ``"simulation_agents"``.

In [ ]:
state_space, df = rp.simulate(params, options)

## How to estimate model parameters.

To estimate model parameters via maximum likelihood, ``respy`` relies on [``estimagic``](https://github.com/OpenSourceEconomics/estimagic), an open-source tool to estimate structural models and more. First, we load the model again, because now we need the data.

In [2]:
params, options, df = rp.get_example_model("kw_94_one")

We make some small adjustments to the options.

In [3]:
options['estimation_tau'] = 100
options['estimation_draws'] = 2000

The dataset is almost in the correct format for the estimation. Note that lagged choices for the first period are missing. As a preliminary fix, we assume that all individuals were in school the previous period which is not unreasonable for individuals with age 16.

In [4]:
df.Lagged_Choice = df.Lagged_Choice.fillna("edu")

Second, we need the criterion function of the model which returns the likelihood value of the data given the model.

In [7]:
options

{'choices': {'edu': {'max': 20, 'start': [10], 'lagged': [1], 'share': [1]}},
 'estimation_draws': 2000,
 'estimation_seed': 500,
 'estimation_tau': 100,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 456}

In [6]:
crit_func = rp.get_crit_func(params, options, df)

/mnt/c/Users/tobia/git/respy/respy/likelihood.py:378: UserWarning: The maximum experience for choice 'a' differ between data, 8, and options, 39. The options are adjusted.
  category=UserWarning,
/mnt/c/Users/tobia/git/respy/respy/likelihood.py:378: UserWarning: The maximum experience for choice 'b' differ between data, 10, and options, 39. The options are adjusted.
  category=UserWarning,
/mnt/c/Users/tobia/git/respy/respy/likelihood.py:365: UserWarning: The initial experience for choice 'edu' differ between data, [ 7  8  9 10 11], and options, [10]. The options are adjusted.
  category=UserWarning,
/mnt/c/Users/tobia/git/respy/respy/likelihood.py:378: UserWarning: The maximum experience for choice 'edu' differ between data, 19, and options, 20. The options are adjusted.
  category=UserWarning,


The criterion function takes the parameters as an input and returns the likelihood value.

In [ ]:
crit_val = crit_func(params)
crit_val

To estimate the model parameters, import the :func:`~estimagic.optimization.optimize.minimize` from ``estimagic``.

In [ ]:
import numpy as np
from estimagic.optimization.optimize import maximize

For ``estimagic``, we need to pass constraints on the parameters in a list containing dictionaries. Each dictionary is a constraint. A constraint includes two components: First, we need to tell ``estimagic`` which paramters we want to constrain. This is achieved by specifying an index location which will be passed to `df.loc`. Then, define the type of the constraint. Here, we only impose the constraint that the shock parameters have to be valid variances and correlations.

In [ ]:
constr = [{"loc": "shocks", "type": "sdcorr"}, {"loc": "delta", "type": "fixed", "value": 0.95}]

Optionally, we can add a column ``"group"`` which is identical to the category column. The estimagic dashboard will then contain one parameter convergence plot per group instead of plotting all parameters in the same figure. Since respy has quite many parameters, this will make the plots much more readable.

In [ ]:
params["group"] = params.index.get_level_values('category')
params['lower'].fillna(-np.inf, inplace=True)
params['upper'].fillna(np.inf, inplace=True)
params

In [ ]:
maximize(crit_func, params, "nlopt_newuoa_bound", db_options={"rollover": 200}, constraints=constr, dashboard=True)

## References

> Keane, M. P. and  Wolpin, K. I. (1994b). [The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence](https://www.minneapolisfed.org/research/staff-reports/the-solution-and-estimation-of-discrete-choice-dynamic-programming-models-by-simulation-and-interpolation-monte-carlo-evidence). *Federal Reserve Bank of Minneapolis*, No. 181.